In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import scipy.io as sio
from scipy import stats
from importlib import reload
from matplotlib import pyplot as plt
import sys
#sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/')
import python_utils_jjm as utils_jjm
import dlc_utils
import caiman
import statsmodels.formula.api as smf

ModuleNotFoundError: No module named 'python_utils_jjm'

In [2]:
reload(utils_jjm)

<module 'python_utils_jjm' from '/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/python_utils_jjm.py'>

In [2]:
#'/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/'
#cnmfe_path = '/volumes/My_Passport/cnmfe_analysis_files/batch_output_files/08-Apr_16_03_01_out.mat'
cnmfe_path = '/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/08-Apr_16_03_01_out.mat'

In [12]:
# align frame inidcies in output file to session 
CNMFE_results = sio.loadmat(cnmfe_path)
#np.shape(CNMFE_results['C'])
#file_array = CNMFE_results['P'][0][0][13]
session_names = []
frames_indicies = []
for session in range(len(file_array)):
    msCam_session = file_array[session][0][0].split('/')[8]
    frames = file_array[session][0][0].split('/')[10]
    frames_idx = frames.strip('frames_').split('_')
    session_names.append(msCam_session)
    frames_indicies.append(frames_idx)

total_frames = np.sum(np.array([(1+int(frames_indicies[session_idx][1]))-int(frames_indicies[session_idx][0]) for session_idx in range(len(session_names))]))

session_key = pd.DataFrame(frames_indicies, index=session_names)
session_key['cumulative_frames'] = np.nan*np.ones(len(session_names))
session_key = session_key.transpose()

final_frames = pd.DataFrame({k:v for k,v in zip(np.unique(np.array(session_names)), [session_key[session].loc[1].values[-1:] for session in np.unique(np.array(session_names))])}, index=['final_frame']).transpose()

final_frames['cumulative_frames'] = np.cumsum(np.array([int(frame) for frame in final_frames['final_frame'].values]))

final_frames

,final_frame,cumulative_frames
H19_M46_S0,1977,1977
H19_M51_S4,2561,4538
H19_M55_S57,3388,7926
H20_M0_S57,4712,12638
H20_M11_S42,5216,17854
H20_M17_S28,4574,22428
H20_M22_S56,4993,27421
H20_M28_S4,4370,31791
H20_M33_S18,5993,37784
H20_M6_S41,4465,42249


In [35]:
# pull out data by session
session = 'H19_M46_S0'
session_idx = np.where(np.array(final_frames.index)==session)[0][0]
frame_end = final_frames.iloc[session_idx]['cumulative_frames']
if session_idx != 0:
    frame_start = final_frames.iloc[session_idx-1]['cumulative_frames']
else:
    frame_start=0
frames=(frame_start, frame_end)
#start/stop indicies of session
frames

(0, 1977)